# Experiment 3 

This notebook contains the code for experiment 3. 

## Loading libraries/packages and the training and test data 

In [1]:
import ssl
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import numpy as np
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
ssl._create_default_https_context = ssl._create_unverified_context
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag
from itertools import chain

import nltk
import sklearn 
import re 
import scipy.stats
import spacy_dbpedia_spotlight


In [3]:
%store -r test_data

In [4]:
train_sents = list(nltk.corpus.conll2002.iob_sents('ned.train'))

## Feature engineering (credits: https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb)

In [5]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'word': word,
        'word.isLower': word.lower(),
        'word.isUpper': word.isupper(),
        'word.isTitle': word.istitle(),
        'word.endsWithPeriod': word.endswith("."),
        'word.isAbbreviation': bool(re.match(r"^((?i)([A-Z])\.)+$", word)),
        'word.containsHyphen': word.find("-"), 
        'word.containsApostrophe': word.find("'"), 
        
    }
    if i > 0:
        prev = sent[i-1][0]
        features.update({
            '-1:word': prev,
            '-1:word.isLower': prev.lower(),
            '-1:word.isUpper': prev.isupper(),
            '-1:word.isTitle': prev.istitle(),
            '-1:word.endsWithPeriod': prev.endswith("."),
            '-1:word.isAbbreviation': bool(re.match(r"^((?i)([A-Z])\.)+$", prev)),
            '-1:word.containsHyphen': prev.find("-"), 
            '-1:word.containsApostrophe': prev.find("'")
        })
    
    if i > 1:
        prev = sent[i-2][0]
        features.update({
            '-2:word': prev,
            '-2:word.isLower': prev.lower(),
            '-2:word.isUpper': prev.isupper(),
            '-2:word.isTitle': prev.istitle(),
            '-2:word.endsWithPeriod': prev.endswith("."), 
            '-2:word.isAbbreviation': bool(re.match(r"^((?i)([A-Z])\.)+$",  prev)),
            '-2:word.containsHyphen': prev.find("-"),
            '-2:word.containsApostrophe': prev.find("'")
        })
    
        
    if i < len(sent)-1:
        next_token = sent[i+1][0]
       
        features.update({
            '+1:word': next_token,
            '+1:word.isLower': next_token.lower(),
            '+1:word.isUpper': next_token.isupper(),
            '+1:word.isTitle': next_token.istitle(),
            '+1:word.endsWithPeriod': next_token.endswith("."), 
            '+1:word.isAbbreviation': bool(re.match(r"^((?i)([A-Z])\.)+$", next_token)),
            '+1:word.containsHyphen': next_token.find("-"),
            '+1:word.containsApostrophe': next_token.find("'")
        })
        
   
    if i < len(sent)-2:
        next_token = sent[i+2][0]
       
        features.update({
            '+2:word': next_token,
            '+2:word.isLower': next_token.lower(),
            '+2:word.isUpper': next_token.isupper(),
            '+2:word.isTitle': next_token.istitle(),
            '+2:word.endsWithPeriod': next_token.endswith("."), 
            '+2:word.isAbbreviation': bool(re.match(r"^((?i)([A-Z])\.)+$", next_token)),
            '+2:word.containsHyphen': next_token.find("-"),
            '+2:word.containsApostrophe': next_token.find("'")
    })

                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

### Training

In [6]:
training_features = [sent2features(s) for s in train_sents]
training_labels = [sent2labels(s) for s in train_sents]

C:\Users\katri\AppData\Local\Temp\ipykernel_37292\3855163242.py:11: DeprecationWarning: Flags not at the start of the expression '^((?i)([A-Z])\\.)+$' but at position 2
  'word.hasabbrev': bool(re.match(r"^((?i)([A-Z])\.)+$", word)),


In [7]:
import sklearn_crfsuite
from sklearn_crfsuite import CRF
from sklearn.metrics import classification_report

crf = CRF(algorithm='l2sgd',
          max_iterations=100,
          all_possible_transitions=False)


In [8]:
crf.fit(training_features, training_labels)


CRF(algorithm='l2sgd', all_possible_transitions=False, max_iterations=100)

### Feature extraction for test data

In [9]:
test_inputs = []
for snippet in test_data:
    sent_features = [sent2features(s) for s in snippet]
    test_inputs.append(sent_features)

## Model predictions

In [10]:
from sklearn.metrics import classification_report
pred_outputs = []
for biography in test_inputs:
    pred = crf.predict(biography)
    pred_outputs.append(pred)

## Formatting predictions

In [11]:
list_of_pred = [] 
for snippet in pred_outputs:
    for sent in snippet:
        for token in sent:
            list_of_pred.append(token)

In [12]:
pred_test_set = test_data
for snippet_index in range(len(pred_test_set)):
    snippet = pred_test_set[snippet_index]
    for sent_index in range(len(snippet)):
        sent = snippet[sent_index]
        pred_test_set[snippet_index][sent_index]= [list(ele) for ele in sent]
    
print(pred_test_set[0][0])

[['MATHAM', 'NNP'], ['(', '('], ['Theodorus', 'NNP'], [')', ')'], [',', ','], ['zoon', 'FW'], ['van', 'FW'], ['Jacobus', 'NNP'], ['Matham', 'NNP'], [',', ','], ['werd', 'NN'], ['in', 'IN'], ['1598', 'CD'], ['te', 'NN'], ['Haarlem', 'NNP'], ['geboren', 'NN'], [',', ','], ['bezocht', 'NN'], ['Italië', 'NNP'], [',', ','], ['en', 'FW'], ['arbeidde', 'FW'], ['aldaar', 'FW'], ['met', 'FW'], ['Cornelis', 'NNP'], ['Blommaent', 'NNP'], [',', ','], ['Natalis', 'NNP'], ['Persyn', 'NNP'], ['en', 'FW'], ['andere', 'RB'], ['Vlaamsche', 'NNP'], ['schilders', 'NNS'], ['aan', 'VBP'], ['de', 'IN'], ['Standbeelden', 'NNP'], ['enz.', 'NN'], ['in', 'IN'], ['de', 'FW'], ['galerij', 'FW'], ['van', 'FW'], ['het', 'NN'], ['paleis', 'NN'], ['Ginstiniani', 'NNP'], ['.', '.']]


In [13]:
k = 0 
i = 0
for snippet_i in range(len(pred_test_set)):
    snippet = pred_test_set[snippet_i]
    for sent_i in range(len(snippet)):
        sent= pred_test_set[snippet_i][sent_i]
        for token_i in range(len(sent)):
            pred_ner = list_of_pred[k]
            pred_test_set[snippet_i][sent_i][token_i].append(pred_ner)
            k+=1
            i += 1
          

## Visualizing predicted results 

In [14]:
def visualize_biography(text):
    import spacy
    from spacy.tokens import Doc, Span
    from spacy import displacy

    nlp = spacy.load("en_core_web_sm")

    vocab = nlp.vocab
    colors = {"B-PER": "#ADD8E6","I-PER": "#ADD8E6", "B-LOC": "#FFD580", "I-LOC": "#FFD580", "B-ORG":"#90EE90","I-ORG":"#90EE90", "B-MISC":"#D8BFD8","I-MISC":"#D8BFD8"}

    docs = []
    for sentence in text:
        without_pos = [[a, c] for [a, b, c] in sentence]
        words, entities = zip(*without_pos)
        doc = Doc(vocab, words=words)
        spans = [Span(doc, i, i + 1, label=ent_type) for i, ent_type in enumerate(entities) if ent_type]
        doc.ents = spans
        docs.append(doc)

    options = {"ents": ["B-PER", "I-PER","B-ORG","I-ORG", "B-LOC","I-LOC","B-MISC","I-MISC"], "colors": colors}
    displacy.render(docs, style="ent", options=options, jupyter=True)

In [15]:
def count_tag(tag,test_set): 
    count = 0 
    for bio in test_set:
        for sentence in bio: 
            for token in sentence:
                if token[2] == tag:
                    count += 1
                    
    return count

In [16]:
print('B-PER', count_tag('B-PER',pred_test_set))
print('I-PER',count_tag('I-PER',pred_test_set))
print('B-LOC',count_tag('B-LOC',pred_test_set))
print('I-LOC',count_tag('I-LOC',pred_test_set))
print('B-ORG',count_tag('B-ORG',pred_test_set))
print('I-ORG',count_tag('I-ORG',pred_test_set))
print('B-MISC',count_tag('B-MISC',pred_test_set))
print('I-MISC',count_tag('I-MISC',pred_test_set))

B-PER 670
I-PER 638
B-LOC 528
I-LOC 73
B-ORG 131
I-ORG 82
B-MISC 388
I-MISC 137


In [17]:
i=0
for bio in pred_test_set:
    print("Bio: ",i)
    visualize_biography(bio)
    i+=1

Bio:  0


Bio:  1


Bio:  2


Bio:  3


Bio:  4


...

Bio:  45


Bio:  46


Bio:  47


Bio:  48


Bio:  49
